## Import Modules

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import cv2
from tqdm.notebook import tqdm
warnings.filterwarnings('ignore')
%matplotlib inline

## Feature extraction

In [2]:
%pip install git+https://github.com/serengil/deepface.git

  Cloning https://github.com/serengil/deepface.git to /tmp/pip-req-build-pvdst3ky
  Running command git clone --filter=blob:none --quiet https://github.com/serengil/deepface.git /tmp/pip-req-build-pvdst3ky
  Resolved https://github.com/serengil/deepface.git to commit c04e51e472be354cd2e5333c3dab9a173876d5ae
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 1.3 MB/s eta 0:00:00ta 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 15.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 58.4 MB/s eta 0:00:00
  Created wheel for deepface: filename=deepface-0.0.91-py3-none-any.whl size=97585 sha256=9d4624fdc64f4aa783f95751e40f15e8485ab0e4b4312210d4b031095174da1f
  Stored in directory: /tmp/pip-ephem-wheel-cache-fypttrxn/wheels/de/c8/76/e5f8bffabb05420b2fe27f9131345b724c402869bc505b6

In [3]:
def get_feature(df, filename = None):
    from deepface import DeepFace
    X = []
    error_images = []
    for image in tqdm(df['image']):
        try:
            embedding_objs = DeepFace.represent(img_path = image, enforce_detection = True)
            X.append(embedding_objs[0]['embedding'])
        except ValueError:
            error_images.append(image)
    X = np.array(X)
    if filename:
        np.save(f'{filename}.npy', X)
    return X

#### Get dataset UTK Face

In [4]:
BASE_DIR = '../input/utkface-new/UTKFace/'

# labels - age, gender, ethnicity
image_paths = []
age_labels = []
gender_labels = []

for filename in tqdm(os.listdir(BASE_DIR)):
    image_path = os.path.join(BASE_DIR, filename)
    temp = filename.split('_')
    age = int(temp[0])
    gender = int(temp[1])
    image_paths.append(image_path)
    age_labels.append(age)
    gender_labels.append(gender)
    
df = pd.DataFrame()
df['image'], df['age'], df['gender'] = image_paths, age_labels, gender_labels
df.head()

  0%|          | 0/23708 [00:00<?, ?it/s]

,image,age,gender
0,../input/utkface-new/UTKFace/26_0_2_2017010402...,26,0
1,../input/utkface-new/UTKFace/22_1_1_2017011223...,22,1
2,../input/utkface-new/UTKFace/21_1_3_2017010500...,21,1
3,../input/utkface-new/UTKFace/28_0_0_2017011718...,28,0
4,../input/utkface-new/UTKFace/17_1_4_2017010322...,17,1


#### Get dataset asia afad

In [5]:
BASE_DIR = '../input/asia-afad/AFAD-Full/'
# labels - age, gender, ethnicity
image_paths = []
age_labels = []
gender_labels = []

for age_dir in os.listdir(BASE_DIR):
    if age_dir.isdigit():
        age = int(age_dir)

        for gender_dir in os.listdir(os.path.join(BASE_DIR, age_dir)):
            if gender_dir.isdigit():
                gender = int(gender_dir)

                for image_file in os.listdir(os.path.join(BASE_DIR, age_dir, gender_dir)):
                    if image_file.endswith('.db'):
                        continue
                    image_path = os.path.join(BASE_DIR, age_dir, gender_dir, image_file)
                    image_paths.append(image_path)
                    if(gender == 111):
                        gender_labels.append(0)
                    if(gender == 112):
                        gender_labels.append(1)
                    age_labels.append(age)
    
df2 = pd.DataFrame()
df2['image'], df2['age'], df2['gender'] = image_paths, age_labels, gender_labels
df2.head()

,image,age,gender
0,../input/asia-afad/AFAD-Full/47/112/855927-0.jpg,47,1
1,../input/asia-afad/AFAD-Full/47/112/855705-0.jpg,47,1
2,../input/asia-afad/AFAD-Full/47/112/854778-0.jpg,47,1
3,../input/asia-afad/AFAD-Full/47/112/854588-0.jpg,47,1
4,../input/asia-afad/AFAD-Full/47/112/857962-0.jpg,47,1


#### Extract feature

In [6]:
# utk-face
get_feature(df, filename='utkface-feature-new')

2024-05-18 16:08:54.783800: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-18 16:08:54.783896: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-18 16:08:54.934986: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


24-05-18 16:09:06 - Directory /root/.deepface created
24-05-18 16:09:06 - Directory /root/.deepface/weights created


  0%|          | 0/23708 [00:00<?, ?it/s]

24-05-18 16:09:07 - vgg_face_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/vgg_face_weights.h5
To: /root/.deepface/weights/vgg_face_weights.h5

  0%|          | 0.00/580M [00:00<?, ?B/s]
  2%|▏         | 13.1M/580M [00:00<00:04, 129MB/s]
  8%|▊         | 48.8M/580M [00:00<00:02, 262MB/s]
 15%|█▍        | 84.4M/580M [00:00<00:01, 303MB/s]
 21%|██        | 120M/580M [00:00<00:01, 323MB/s] 
 27%|██▋       | 156M/580M [00:00<00:01, 335MB/s]
 33%|███▎      | 191M/580M [00:00<00:01, 341MB/s]
 39%|███▉      | 228M/580M [00:00<00:01, 348MB/s]
 45%|████▌     | 263M/580M [00:00<00:00, 349MB/s]
 51%|█████▏    | 298M/580M [00:00<00:00, 351MB/s]
 58%|█████▊    | 334M/580M [00:01<00:00, 353MB/s]
 64%|██████▍   | 370M/580M [00:01<00:00, 353MB/s]
 70%|██████▉   | 406M/580M [00:01<00:00, 352MB/s]
 76%|███████▌  | 441M/580M [00:01<00:00, 351MB/s]
 82%|████████▏ | 477M/580M [00:01<00:00, 351MB/s]
 88%|████████▊ | 513M/580M [00:01<00:00, 353MB/s]
100%|██████████| 580M/580M [00:01<00:00, 340MB

KeyboardInterrupt: 

In [7]:
# asia-afad
X1 = get_feature(df2.iloc[:82750])
X2 = get_feature(df2.iloc[82750:])
X = np.concatenante((X1, X2), axis=0)
np.save('asia-afad-feature.npy', X)

  0%|          | 0/82750 [00:00<?, ?it/s]

KeyboardInterrupt: 